[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)


# Results:

By Friday, April 29 at 11:59PM, you must submit a draft of your results for at least one research question (we recommend trying to have a draft of both done by this time). If (and only if) you address all the criteria in the corresponding Results section above, you will receive full credit on the
checkpoint.
- MULTIPLE HYPOTHESIS TESTING: 
  - Summarize and interpret the results from the hypothesis tests themselves.
  - For the two correction methods you chose, clearly explain what kind of error rate is being controlled by each one
- CAUSAL INFERENCE:
  - Summarize and interpret your results, providing a clear statement about causality (or a lack thereof) including any assumptions necessary.
  - Where possible, discuss the uncertainty in your estimate and/or the evidence against the hypotheses you are investigating.
  
You are free to change your results section or add (or remove) content between the checkpoint and your final submission. Course staff will not provide any feedback on the research question checkpoint.

## Multiple Hypothesis Testing

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import abline_plot
import datetime
import math

import matplotlib.pyplot as plt

In [7]:
import random 
df = pd.read_csv('https://raw.githubusercontent.com/haroldcha/gunviolencedata/master/gun_violence.csv')

In [8]:
# reminder of cleaned dataset
df

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,gun_stolen,...,participant_status,participant_type,state_house_district,state_senate_district,datetime,year,state_pop,city_pop,area (sq. mi),pop_density
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,14.0,NaN,...,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,NaN,NaN,2013-01-01,2013,12776309,NaN,46058,277.396088
1,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,43.0,NaN,...,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,62.0,35.0,2013-01-01,2013,38260787,85863.0,163707,233.715034
2,2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,9.0,0::Unknown||1::Unknown,...,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,56.0,13.0,2013-01-01,2013,11576684,63735.0,44828,258.246721
3,3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,6.0,NaN,...,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,40.0,28.0,2013-01-05,2013,5269035,345613.0,104100,50.615130
4,4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,6.0,0::Unknown||1::Unknown,...,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,62.0,27.0,2013-01-07,2013,9843336,279244.0,53821,182.890247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,NaN,0::Unknown,...,"0::Unharmed, Arrested",0::Subject-Suspect,NaN,NaN,2018-03-31,2018,4659690,NaN,51843,89.880794
239673,239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,4.0,0::Unknown,...,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,23.0,31.0,2018-03-31,2018,4659690,NaN,51843,89.880794
239674,239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,2.0,0::Unknown,...,0::Injured,0::Victim,85.0,7.0,2018-03-31,2018,4659690,NaN,51843,89.880794
239675,239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,9.0,0::Unknown,...,0::Killed,0::Victim,149.0,17.0,2018-03-31,2018,28628666,2318573.0,268601,106.584361


In [9]:
def test_statistic(df, column):
    means_table = df.groupby(column).mean()
    if 1 not in means_table.index:
        diff = 0 - means_table.loc[0].values
    elif 0 not in means_table.index:
        diff = means_table.loc[1].values - 0
    else:
        diff = means_table.loc[1].values - means_table.loc[0].values
    
    return diff

In [10]:
def simulated_test_stat(df, column):
    shuffled_labels = df.sample(frac=1, replace=True)[column]
    df["shuffled_labels"] = shuffled_labels.to_numpy()
    df = df.drop(column,axis=1)
    
    return test_statistic(df, 'shuffled_labels')   

In [11]:
def p_value(df, column, observed_difference, comparison):
    differences = []

    repetitions = 5000
    for i in np.arange(repetitions):
        new_difference = simulated_test_stat(df, column)
        differences = np.append(differences, new_difference) 
    if comparison=="greater":
        #indicates that higher values of the difference favor the alternative hypothesis that the treatment incidents were higher on average
        print(differences, observed_difference)
        empirical_p = np.count_nonzero(differences >= observed_difference) / repetitions
    else:
        #indicates that lower values of the difference favor the alternative hypothesis that the treatment incidents were lower on average
        empirical_p = np.count_nonzero(differences <= observed_difference) / repetitions  
    return empirical_p

### 1. Are more populated cities associated with deadlier instances of gun violence?

Null hypothesis: In the U.S., the distribution of deadly instances of gun violence (according to our definition of deadly above) is the same for cities that are populated and cities that are not. The difference in the sample is due to chance.

Alternative hypothesis: In the U.S., the instances of gun violence are deadlier for cities that are populated vs. cities that are not. 



In [12]:
most_populous = df.loc[df['city_or_county'].isin(['New York','Los Angeles','Chicago','Houston','Phoenix'])]
most_populous['populous?'] = 1
least_populous = df.loc[df['city_or_county'].isin(['Lakewood', 'Troy', 'Saginaw', 'Niagara Falls', 'Chaleston'])]
least_populous['populous?'] = 0
populous = pd.concat([most_populous, least_populous])
populous_df = populous.groupby(['city_or_county', 'populous?']).sum()
populous_df['num_incidents'] = populous.groupby(['city_or_county', 'populous?']).count()['incident_id'].values
populous_df['num_victims'] = populous_df['n_killed'].values + populous_df['n_injured'].values
populous_df = populous_df.loc[:,['num_incidents', 'num_victims']]
populous_df = populous_df.reset_index('populous?')
populous_df

/tmp/ipykernel_176/1572256797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_populous['populous?'] = 1
/tmp/ipykernel_176/1572256797.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  least_populous['populous?'] = 0


,populous?,num_incidents,num_victims
city_or_county,,,
Chicago,1,10814,12531
Houston,1,2501,2400
Lakewood,0,113,85
Los Angeles,1,1066,1189
New York,1,377,327
Niagara Falls,0,182,75
Phoenix,1,973,886
Saginaw,0,242,218
Troy,0,167,89


In [13]:
victim_populous = populous_df.loc[:,['populous?', 'num_victims']]
test_victim_populous = test_statistic(victim_populous, 'populous?')
p_value(victim_populous, 'populous?', test_victim_populous, 'greater')

[-1894.1         1918.64285714  4541.33333333 ... -2196.5
  2935.85        2440.        ] [3349.85]


0.07

### 2. Are more populated cities associated with more instances of gun violence?

Null hypothesis: In the U.S., the distribution of instances of gun violence is the same for cities that are populated and cities that are not. The difference in the sample is due to chance.

Alternative hypothesis: In the U.S., the number of instances of gun violence (according to our definition of deadly above) is higher for cities that are populated vs. cities that are not. 

In [14]:
incidents_populous = populous_df.loc[:,['populous?', 'num_incidents']]
test_incidents_populous = test_statistic(incidents_populous, 'populous?')
p_value(incidents_populous, 'populous?', test_incidents_populous, 'greater')

[ 2249.8        -2851.4        -1566.78571429 ... -1420.21428571
 -1933.         -2583.2       ] [2970.2]


0.0564

### 3. Are more densely populated states associated with deadlier instances of gun violence?

Null hypothesis: In the U.S., the distribution of deadly instances of gun violence (according to our definition of deadly above) is the same for cities that are densely populated and cities that are not. The difference in the sample is due to chance.

Alternative hypothesis: In the U.S., the instances of gun violence are deadlier for cities that are densely populated vs. cities that are not. 

In [11]:
df

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,gun_stolen,...,participant_status,participant_type,state_house_district,state_senate_district,datetime,year,state_pop,city_pop,area (sq. mi),pop_density
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,14.0,NaN,...,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,NaN,NaN,2013-01-01,2013,12776309,NaN,46058,277.396088
1,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,43.0,NaN,...,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,62.0,35.0,2013-01-01,2013,38260787,85863.0,163707,233.715034
2,2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,9.0,0::Unknown||1::Unknown,...,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,56.0,13.0,2013-01-01,2013,11576684,63735.0,44828,258.246721
3,3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,6.0,NaN,...,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,40.0,28.0,2013-01-05,2013,5269035,345613.0,104100,50.615130
4,4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,6.0,0::Unknown||1::Unknown,...,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,62.0,27.0,2013-01-07,2013,9843336,279244.0,53821,182.890247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,NaN,0::Unknown,...,"0::Unharmed, Arrested",0::Subject-Suspect,NaN,NaN,2018-03-31,2018,4659690,NaN,51843,89.880794
239673,239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,4.0,0::Unknown,...,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,23.0,31.0,2018-03-31,2018,4659690,NaN,51843,89.880794
239674,239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,2.0,0::Unknown,...,0::Injured,0::Victim,85.0,7.0,2018-03-31,2018,4659690,NaN,51843,89.880794
239675,239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,9.0,0::Unknown,...,0::Killed,0::Victim,149.0,17.0,2018-03-31,2018,28628666,2318573.0,268601,106.584361


In [15]:
most_dense = df.loc[df['city_or_county'].isin(['New York','San Francisco','Boston','Miami','Chicago'])]
most_dense['dense?'] = 1
least_dense = df.loc[df['city_or_county'].isin(['Anchorage','Augusta','Norman','Chesapeake','Columbus'])]
least_dense['dense?'] = 0
dense = pd.concat([most_dense, least_dense])
dense_df = dense.groupby(['city_or_county', 'dense?']).sum()
dense_df['num_incidents'] = dense.groupby(['city_or_county', 'dense?']).count()['incident_id'].values
dense_df['num_victims'] = dense_df['n_killed'].values + dense_df['n_injured'].values
dense_df = dense_df.loc[:,['num_incidents', 'num_victims']]
dense_df = dense_df.reset_index('dense?')
dense_df

/tmp/ipykernel_176/1010585100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_dense['dense?'] = 1
/tmp/ipykernel_176/1010585100.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  least_dense['dense?'] = 0


,dense?,num_incidents,num_victims
city_or_county,,,
Anchorage,0,469,298
Augusta,0,362,251
Boston,1,1737,762
Chesapeake,0,141,127
Chicago,1,10814,12531
Columbus,0,2252,1883
Miami,1,846,922
New York,1,377,327
Norman,0,50,36


In [16]:
victims_dense = dense_df.loc[:,['dense?', 'num_victims']]
test_victim_dense = test_statistic(victims_dense, 'dense?')
p_value(victims_dense, 'dense?', test_victim_dense, 'greater')

[ 2036.25        2609.58333333  1527.5        ...  2315.83333333
  2791.6        -2214.76190476] [2472.]


0.1942

### 4. Are more densely populated states associated with more instances of gun violence?

Null hypothesis: In the U.S., the distribution of instances of gun violence is the same for cities that are densely populated and cities that are not. The difference in the sample is due to chance.

Alternative hypothesis: In the U.S., the number of instances of gun violence is higher for cities that are densely populated vs. cities that are not. 

In [17]:
incidents_dense = dense_df.loc[:,['dense?', 'num_victims']]
test_incidents_dense = test_statistic(incidents_dense, 'dense?')
p_value(incidents_dense, 'dense?', test_incidents_dense, 'greater')

[ 2287.61904762   808.57142857  6214.375      ...  1858.75
 -1867.2         2606.        ] [2472.]


0.1914

### 5. Are crimes getting less violent over time (number of casualties)?

Null hypothesis: In the U.S., the distribution of instances of deadly gun violence is the same before as it was after. The difference in the sample is due to chance.

Alternative hypothesis: In the U.S., the number of instances of deadly gun is higher before than it was after. 

if before == 1, we predict a greater number of casualties. if before == 0, it would be less because the number went down. therefore mean(1) - mean(0) would be positive. we want to see how likely it is to observe values more extreme (greater than this)

In [15]:
df

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,gun_stolen,...,participant_status,participant_type,state_house_district,state_senate_district,datetime,year,state_pop,city_pop,area (sq. mi),pop_density
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,14.0,NaN,...,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,NaN,NaN,2013-01-01,2013,12776309,NaN,46058,277.396088
1,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,43.0,NaN,...,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,62.0,35.0,2013-01-01,2013,38260787,85863.0,163707,233.715034
2,2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,9.0,0::Unknown||1::Unknown,...,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,56.0,13.0,2013-01-01,2013,11576684,63735.0,44828,258.246721
3,3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,6.0,NaN,...,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,40.0,28.0,2013-01-05,2013,5269035,345613.0,104100,50.615130
4,4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,6.0,0::Unknown||1::Unknown,...,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,62.0,27.0,2013-01-07,2013,9843336,279244.0,53821,182.890247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,NaN,0::Unknown,...,"0::Unharmed, Arrested",0::Subject-Suspect,NaN,NaN,2018-03-31,2018,4659690,NaN,51843,89.880794
239673,239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,4.0,0::Unknown,...,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,23.0,31.0,2018-03-31,2018,4659690,NaN,51843,89.880794
239674,239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,2.0,0::Unknown,...,0::Injured,0::Victim,85.0,7.0,2018-03-31,2018,4659690,NaN,51843,89.880794
239675,239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,9.0,0::Unknown,...,0::Killed,0::Victim,149.0,17.0,2018-03-31,2018,28628666,2318573.0,268601,106.584361


In [28]:
before = df.loc[df['year'].isin([2013, 2014, 2015])]
before['before?'] = 1
after = df.loc[df['year'].isin([2016, 2017, 2018])]
after['before?'] = 0
time = pd.concat([after, before])
time_df = time.groupby(['year', 'before?']).sum()
time_df['num_victims'] = time_df['n_killed'].values + time_df['n_injured'].values
time_df = time_df.loc[:,['num_victims']]
time_df = time_df.reset_index('before?')
time_df

/tmp/ipykernel_176/762332063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before['before?'] = 1
/tmp/ipykernel_176/762332063.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after['before?'] = 0


,before?,num_victims
year,,
2013,1,1296
2014,1,35559
2015,1,40451
2016,0,45646
2017,0,46214
2018,0,9704


In [29]:
test_time = test_statistic(time_df, 'before?')
p_value(time_df, 'before?', test_time, 'greater')

[  4244.          29811.66666667 -19001.2        ... -10770.25
  25322.66666667 -25322.66666667] [-8086.]


0.6584

### Corrections

In [ ]:
def report_results(predicted_discoveries, truth):
    """
    Produces a dictionary with counts for the true positives, true negatives,
    false negatives, and false positives from the input `predicted_discoveries`
    and `truth` arrays.
    
    Inputs:
      predicted discoveries: array of 0/1 values where 1 indicates a "discovery".
      truth: array of 0/1 values where 1 indicates a draw from the alternative.
    
    Outputs: a dictionary of TN, TP, FN, and FP counts.
    """   
    
    # populate the following dictionary with counts (NOT rates)
    TP_count = np.count_nonzero([predicted_discoveries * truth])
    TN_count = np.count_nonzero([(predicted_discoveries-1) * (truth-1)])
    FP_count = np.count_nonzero([predicted_discoveries * (truth-1)])
    FN_count = np.count_nonzero([(predicted_discoveries-1) * truth])
    
    results_dictionary = {"TN_count": TN_count,
                          "TP_count": TP_count,
                          "FN_count": FN_count,
                          "FP_count": FP_count,
                         }
    
    return results_dictionary

In [4]:
def bonferroni(df, column, alpha):
    """
    Returns decisions on p-values using the Bonferroni correction.
    
    Inputs:
        p_values: array of p-values
        alpha_total: desired family-wise error rate (FWER = P(at least one false discovery))
    
    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    p_values = []
    for i in range(1000):
        test = test_statistic(df, column)
        p = p_value(df, column, test, 'greater')
        p_values.append(p_values)
    
    decisions = (p_values <= alpha/len(p_values))
    return decisions

In [ ]:
bonferroni_decisions = bonferroni(p_values, alpha)

results = report_results(bonferroni_decisions,true_values)
print()
print_false_discovery_fraction(results)
print()

In [ ]:
def benjamini_hochberg(df, column, alpha):
    """
    Returns decisions on p-values using Benjamini-Hochberg.
    
    Inputs:
        p_values: array of p-values
        alpha: desired FDR (FDR = E[# false positives / # positives])
    
    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    p_values = []
    for i in range(1000):
        test = test_statistic(df, column)
        p = p_value(df, column, test, 'greater')
        p_values.append(p_values)
        
    p_list = np.array(p_values)
    p_list.sort()
    k = np.arange(len(p_values)) + 1
    pk = k*alpha/len(p_values)
    boolean_array = list(p_list <= pk)
    p_max = np.max(p_list[boolean_array])
    decisions = (p_values <= p_max)
    return decisions

In [ ]:
bh_decisions = benjamini_hochberg(p_values, alpha)

bh_results = report_results(bh_decisions,true_values)
print()

print_false_discovery_fraction(bh_results)

# Citations

- https://inferentialthinking.com/chapters/12/1/AB_Testing.html
- 'lab01', Data 102 https://data102.datahub.berkeley.edu/user/doud/notebooks/sp22/labs/lab01/lab01.ipynb